In [1]:
# -*- coding: utf-8 -*-
# This is a report using the data from IQAASL.
# IQAASL was a project funded by the Swiss Confederation
# It produces a summary of litter survey results for a defined region.
# These charts serve as the models for the development of plagespropres.ch
# The data is gathered by volunteers.
# Please remember all copyrights apply, please give credit when applicable
# The repo is maintained by the community effective January 01, 2022
# There is ample opportunity to contribute, learn and teach
# contact dev@hammerdirt.ch

# Dies ist ein Bericht, der die Daten von IQAASL verwendet.
# IQAASL war ein von der Schweizerischen Eidgenossenschaft finanziertes Projekt.
# Es erstellt eine Zusammenfassung der Ergebnisse der Littering-Umfrage für eine bestimmte Region.
# Diese Grafiken dienten als Vorlage für die Entwicklung von plagespropres.ch.
# Die Daten werden von Freiwilligen gesammelt.
# Bitte denken Sie daran, dass alle Copyrights gelten, bitte geben Sie den Namen an, wenn zutreffend.
# Das Repo wird ab dem 01. Januar 2022 von der Community gepflegt.
# Es gibt reichlich Gelegenheit, etwas beizutragen, zu lernen und zu lehren.
# Kontakt dev@hammerdirt.ch

# Il s'agit d'un rapport utilisant les données de IQAASL.
# IQAASL était un projet financé par la Confédération suisse.
# Il produit un résumé des résultats de l'enquête sur les déchets sauvages pour une région définie.
# Ces tableaux ont servi de modèles pour le développement de plagespropres.ch
# Les données sont recueillies par des bénévoles.
# N'oubliez pas que tous les droits d'auteur s'appliquent, veuillez indiquer le crédit lorsque cela est possible.
# Le dépôt est maintenu par la communauté à partir du 1er janvier 2022.
# Il y a de nombreuses possibilités de contribuer, d'apprendre et d'enseigner.
# contact dev@hammerdirt.ch

# sys, file and nav packages:
import datetime as dt
from datetime import date, datetime, time
from babel.dates import format_date, format_datetime, format_time, get_month_names
import locale

# math packages:
import pandas as pd
import numpy as np
from scipy import stats
from math import pi

# charting:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import ticker
from matplotlib.ticker import MultipleLocator
import seaborn as sns

# the module that has all the methods for handling the data
import resources.featuredata as featuredata

# home brew utitilties
import resources.chart_kwargs as ck
import resources.sr_ut as sut

# images and display
from IPython.display import Markdown as md
from myst_nb import glue

# chart style
sns.set_style("whitegrid")

# colors for gradients
cmap2 = ck.cmap2
colors_palette = ck.colors_palette
bassin_pallette = featuredata.bassin_pallette


# border and row shading fro tables
a_color = "saddlebrown"
table_row = "saddlebrown"

## !! Begin Note book variables !!
# There are two language variants: german and english
# change both: date_lang and language
date_lang =  'de_DE.utf8'
locale.setlocale(locale.LC_ALL, date_lang)

# the date format of the survey data is defined in the module
date_format = featuredata.date_format

# the language setting use lower case: en or de
# changing the language may require changing the unit label
language = "de"
unit_label = "p/100 m"

# the standard date format is "%Y-%m-%d" if your date column is
# not in this format it will not work.
# these dates cover the duration of the IQAASL project
start_date = "2020-03-01"
end_date ="2021-05-31"
start_end = [start_date, end_date]

# the fail rate used to calculate the most common codes is
# 50% it can be changed:
fail_rate = 50

# Changing these variables produces different reports
# Call the map image for the area of interest
bassin_map = "resources/maps/survey_areas/aare_scaled.jpeg"

# the label for the aggregation of all data in the region
top = "Alle Erhebungsgebiete"

# define the feature level and components
# the feature of interest is the Aare (aare) at the river basin (river_bassin) level.
# the label for charting is called 'name'
this_feature = {'slug':'all', 'name':"Alle Erhebungsgebiete", 'level':'all'}

# these are the smallest aggregated components
# choices are water_name_slug=lake or river, city or location at the scale of a river bassin 
# water body or lake maybe the most appropriate
this_level = 'river_bassin'

# !! End note book variables !!

## data
# Survey location details (GPS, city, land use)
dfBeaches = pd.read_csv("resources/beaches_with_land_use_rates.csv")
# set the index of the beach data to location slug
dfBeaches.set_index("slug", inplace=True)

# Survey dimensions and weights
dfDims = pd.read_csv("resources/corrected_dims.csv")

# code definitions
dxCodes = pd.read_csv("resources/codes_with_group_names")
dxCodes.set_index("code", inplace=True)

# columns that need to be renamed. Setting the language will automatically
# change column names, code descriptions and chart annotations
columns={"% to agg":"% agg", "% to recreation": "% recreation", "% to woods":"% woods", "% to buildings":"% buildings", "p/100m":"p/100 m"}

# key word arguments to construct feature data
# !Note the water type allows the selection of river or lakes
# if None then the data is aggregated together. This selection
# is only valid for survey-area reports or other aggregated data
# that may have survey results from both lakes and rivers.
fd_kwargs ={
    "filename": "resources/checked_sdata_eos_2020_21.csv",
    "feature_name": this_feature['slug'], 
    "feature_level": this_feature['level'], 
    "these_features": this_feature['slug'], 
    "component": this_level, 
    "columns": columns, 
    "language": 'de', 
    "unit_label": unit_label, 
    "fail_rate": fail_rate,
    "code_data":dxCodes,
    "date_range": start_end,
    "water_type": None,    
}

fdx = featuredata.Components(**fd_kwargs)

# call the reports and languages
fdx.adjustForLanguage()
fdx.makeFeatureData()
fdx.locationSampleTotals()
fdx.makeDailyTotalSummary()
fdx.materialSummary()
fdx.mostCommon()
# !this is the feature data!
fd = fdx.feature_data

# the period data is all the data that was collected
# during the same period from all the other locations
# not included in the feature data for a survey area
# or river bassin the parent and feature level are the
# the same.
period_kwargs = {
    "period_data": fdx.period_data,
    "these_features": this_feature['slug'],
    "feature_level":this_feature['level'],
    "feature_parent":this_feature['slug'],
    "parent_level": this_feature['level'],
    "period_name": top,
    "unit_label": unit_label,
    "most_common": fdx.most_common.index
}
period_data = featuredata.PeriodResults(**period_kwargs)

# the rivers are considered separately
# select only the results from rivers
fd_kwargs.update({"water_type":"r"})
fdr = featuredata.Components(**fd_kwargs)
fdr.makeFeatureData()
fdr.adjustForLanguage()
fdr.locationSampleTotals()
fdr.makeDailyTotalSummary()
fdr.materialSummary()
fdr.mostCommon()

# collects the summarized values for the feature data
# use this to generate the summary data for the survey area
# and the section for the rivers
admin_kwargs = {
    "data":fd,
    "dims_data":dfDims,
    "label": this_feature["name"],
    "feature_component": this_level,
    "date_range":start_end,
    **{"dfBeaches":dfBeaches}
}
admin_details = featuredata.AdministrativeSummary(**admin_kwargs)
admin_summary = admin_details.summaryObject()
admin_r_details = featuredata.AdministrativeSummary(data=fdr.feature_data, dims_data=dfDims, label=this_feature["name"], feature_component=this_level, date_range=start_end, **{"dfBeaches":dfBeaches})

admin_kwargs.update({"data":fdr.feature_data})
admin_r_details = featuredata.AdministrativeSummary(**admin_kwargs)
admin_r_summary = admin_r_details.summaryObject()

In [2]:
fd.water_name_slug.unique()

array(['lac-leman', 'lago-di-lugano', 'cassarate', 'ticino',
       'lago-maggiore', 'aarenidau-buren-kanal', 'limmat', 'emme',
       'zugersee', 'neuenburgersee', 'thunersee', 'aare',
       'quatre-cantons', 'bielersee', 'sihl', 'walensee', 'zurichsee',
       'rhone', 'schuss', 'brienzersee', 'jona', 'reuss', 'linthkanal',
       'maggia', 'la-thiele', 'dorfbach', 'escherkanal', 'seez'],
      dtype=object)

(gisoutput)=
# GIS output

In [3]:
# this gets all the data for the project
land_use_kwargs = {
    "data": period_data.period_data,
    "index_column":"loc_date",
    "these_features": this_feature['slug'],
    "feature_level":this_level   
}

# the landuse profile of the project
project_profile = featuredata.LandUseProfile(**land_use_kwargs).byIndexColumn()

# update the kwargs for the feature data
land_use_kwargs.update({"data":fdx.feature_data})

# build the landuse profile of the feature
feature_profile = featuredata.LandUseProfile(**land_use_kwargs)

# this is the component features of the report
feature_landuse = feature_profile.featureOfInterest()

In [4]:
# the dimensional data
dims_table = admin_details.dimensionalSummary()

# a method to update the place names from slug to proper name
name_map = featuredata.river_basin_de

# the order in which they are charted
name_order = list(name_map.keys())

# sort by quantity
dims_table.sort_values(by=["quantity"], ascending=False, inplace=True)

# translating column names
dims_table.rename(columns=featuredata.dims_table_columns_de, inplace=True)

# the values in these columns need formating to swiss spec
thousands_separated = ["Fläche (m2)", "Länge (m)", "Erhebungen", "Objekte (St.)"]
replace_decimal = ["Plastik (Kg)", "Gesamtgewicht (Kg)"]
dims_table["Plastik (Kg)"] = dims_table["Plastik (Kg)"]/1000
dims_table.reset_index(inplace=True)
dims_table["river_bassin"] = dims_table.river_bassin.map(lambda x: featuredata.updatePlaceNames(x=x, a_map=name_map))

dims_table

,river_bassin,Gesamtgewicht (Kg),Plastik (Kg),Fläche (m2),Länge (m),Erhebungen,Objekte (St.)
0,Alle Erhebungsgebiete,305.507,94.213346,96616.35,19722.0,386.0,54744.0
1,Rhone,151.309,45.967375,25986.25,4911.0,106.0,28454.0
2,Aare,71.976,31.448170,37017.80,7971.0,140.0,13847.0
3,Linth,35.961,12.772621,25637.80,5323.0,112.0,9412.0
4,Ticino,46.261,4.025180,7974.50,1517.0,28.0,3031.0


### The sample location layer

The median sample total in p/100 m for each location is figured and the lat, lon are attached. Included identifying information:

1. city
2. survey area
3. quantity
4. number of samples
5. the feature name

In [5]:
# the sample totals of the parent feautre
dx = period_data.parentSampleTotals(parent=False)

# sample totals of the feature data
sample_totals = fdx.sample_totals
these_beaches = admin_details.df_beaches.loc[sample_totals.location.unique()]

# for each location or level sum the quantity and get the median survey value
agg_columns = {"quantity": "sum", "p/100 m": "median", "loc_date": "nunique"}
columns = ["river_bassin", "location"]

location_layer = sample_totals.groupby(columns, as_index=False).agg(agg_columns)

lat_lon = these_beaches[["latitude", "longitude"]]
cities = these_beaches["city"]
feature_map = admin_details.makeLocationFeatureMap()

location_layer["lat"] = location_layer.location.map(lambda x: lat_lon.loc[x, "latitude"])
location_layer["lon"] = location_layer.location.map(lambda x: lat_lon.loc[x, "longitude"])
location_layer["city"] = location_layer.location.map(lambda x: cities.loc[x])
location_layer["feature"] = location_layer.location.map(lambda x: feature_map.loc[x])
location_layer['samples'] = location_layer.loc_date

location_layer.drop("loc_date", axis=1, inplace=True)

location_layer.to_csv("resources/output/gis/location_layer.csv", index=False)

location_layer.head()

,river_bassin,location,quantity,p/100 m,lat,lon,city,feature,samples
0,aare,aare-limmatspitz,70,60.0,47.501060,8.237371,Gebenstorf,aare,1
1,aare,aare-port,99,253.0,47.116170,7.269550,Port,aarenidau-buren-kanal,1
2,aare,aare-solothurn-lido-strand,27,244.0,47.196949,7.521643,Solothurn,aare,1
3,aare,aare_bern_gerberm,134,363.0,46.989363,7.452098,Bern,aare,1
4,aare,aare_bern_scheurerk,4,12.0,46.970967,7.452586,Bern,aare,1


### The survey area layer

The survey area layer is the aggregate of the sample totals grouped by survey area. The latitude and longitude of the point is 
determined by the first record with a matching survey area. The attributes are assigned to a polygon that includes all the municipalities
in the survey area. The median sample total for the survey area is reported as well as

1. quantity
2. number of samples
3. survey area label

In [6]:
surveyarea_layer = sample_totals.groupby("river_bassin", as_index=False).agg(agg_columns)

rbassins =  these_beaches[["river_bassin", "latitude", "longitude"]].drop_duplicates("river_bassin")
rbassins.set_index("river_bassin", inplace=True, drop=True)

surveyarea_layer["lat"] = surveyarea_layer.river_bassin.map(lambda x: rbassins.loc[x, "latitude"])
surveyarea_layer["lon"] = surveyarea_layer.river_bassin.map(lambda x: rbassins.loc[x, "longitude"])
surveyarea_layer["samples"] = surveyarea_layer.loc_date

surveyarea_layer.drop("loc_date", axis=1, inplace=True)

surveyarea_layer.to_csv("resources/output/gis/surveyarea_layer.csv", index=False)

surveyarea_layer.head()

,river_bassin,quantity,p/100 m,lat,lon,samples
0,aare,13847,143.5,47.501060,8.237371,140
1,linth,9412,131.5,47.220989,8.940365,112
2,rhone,28454,442.5,46.447216,6.859612,106
3,ticino,3031,160.5,46.153882,8.768480,28


### The water feature layer

The water feature layer is the aggregate of the sample totals grouped by water feature area.

In [7]:
water_feature_map = admin_details.makeLocationFeatureMap()
water_feature_layer = sample_totals.copy()
water_feature_layer["water_feature"] = water_feature_layer.location.map(lambda x: water_feature_map.loc[x])

wfeatures =  these_beaches[["water_name_slug", "latitude", "longitude"]].drop_duplicates("water_name_slug")
wfeatures.set_index("water_name_slug", inplace=True, drop=True)

water_feature_layer = water_feature_layer.groupby("water_feature", as_index=False).agg(agg_columns)
water_feature_layer["lat"] = water_feature_layer.water_feature.map(lambda x: wfeatures.loc[x, "latitude"])
water_feature_layer["lon"] = water_feature_layer.water_feature.map(lambda x: wfeatures.loc[x, "longitude"])
water_feature_layer["samples"] = water_feature_layer.loc_date

water_feature_layer.drop("loc_date", axis=1, inplace=True)

water_feature_layer.to_csv("resources/output/gis/water_feature_layer.csv", index=False)

water_feature_layer.head()

,water_feature,quantity,p/100 m,lat,lon,samples
0,aare,478,49.0,47.501060,8.237371,15
1,aarenidau-buren-kanal,155,102.0,47.116170,7.269550,3
2,bielersee,4477,340.0,47.038398,7.108311,38
3,brienzersee,974,334.0,46.690283,7.898592,5
4,cassarate,54,75.0,46.002411,8.961477,1


### The municipal area layer

The municipal layer is the aggregate of the sample totals grouped by city. The latitude and longitude of the point is 
determined by the first record with a matching city. The attributes are assigned to a polygon of that city. The median sample total for the survey area is reported as well as:
1. quantity
2. number of samples
3. survey area label

In [8]:
# calling the components class on updated keywords
# aggregatest to the city level
l_and_r = these_beaches[["water_name_slug", "water"]].drop_duplicates().set_index("water_name_slug", drop=True)
fd_kwargs_city ={
    "filename": "resources/checked_sdata_eos_2020_21.csv",
    "feature_name": this_feature['slug'], 
    "feature_level": this_feature['level'], 
    "these_features": this_feature['slug'], 
    "component": "city", 
    "columns": columns, 
    "language": 'de', 
    "unit_label": unit_label, 
    "fail_rate": fail_rate,
    "code_data":dxCodes,
    "date_range": start_end,
    "water_type": None,    
}

fdc = featuredata.Components(**fd_kwargs_city)

fdc.adjustForLanguage()
fdc.makeFeatureData()
fdc.locationSampleTotals()
fdc.makeDailyTotalSummary()
fdc.materialSummary()
fdc.mostCommon()
# !this is the feature data!
data = fdc.sample_totals.copy()
data["river_bassin"] = data.location.map(lambda x : these_beaches["river_bassin"].loc[x])
data["feature"] = data.location.map(lambda x: feature_map.loc[x])
data["type"] = data.feature.map(lambda x: l_and_r.loc[x, "water"])

In [9]:
municipal_layer = data.groupby(["city", "river_bassin", "feature", "type"], as_index=False).agg(agg_columns)

city_gps =  these_beaches[["city", "latitude", "longitude"]].drop_duplicates("city")
city_gps.set_index("city", inplace=True, drop=True)

municipal_layer["lat"] = municipal_layer.city.map(lambda x: city_gps.loc[x, "latitude"])
municipal_layer["lon"] = municipal_layer.city.map(lambda x: city_gps.loc[x, "longitude"])
municipal_layer["type"] = municipal_layer.feature.map(lambda x: l_and_r.loc[x, "water"])
municipal_layer["samples"] = municipal_layer.loc_date
# location_layer["feature"] = location_layer.location.map(lambda x: feature_map.loc[x])

municipal_layer.drop("loc_date", axis=1, inplace=True)

municipal_layer.to_csv("resources/output/gis/municipal_layer_all.csv", index=False)

label_layer = municipal_layer.drop_duplicates("city")
label_layer.to_csv("resources/output/gis/municipal_label_layer.csv", index=False)

municipal_layer.head()

,city,river_bassin,feature,type,quantity,p/100 m,lat,lon,samples
0,Aarau,aare,aare,r,7,35.0,47.405669,8.066018,1
1,Allaman,rhone,lac-leman,l,631,716.0,46.463919,6.385732,3
2,Ascona,ticino,lago-maggiore,l,433,172.0,46.153882,8.768480,5
3,Beatenberg,aare,thunersee,l,104,242.5,46.684386,7.794768,2
4,Bellinzona,ticino,ticino,r,187,121.0,46.200625,9.015853,4


### The alps and jura

In [10]:
# key word arguments to construct feature data
# !Note the water type allows the selection of river or lakes
# if None then the data is aggregated together. This selection
# is only valid for survey-area reports or other aggregated data
# that may have survey results from both lakes and rivers.

# the alpes surveys happened after the end of sampling
# the standard date format is "%Y-%m-%d" if your date column is
# not in this format it will not work.
# these dates cover the duration of the IQAASL project
start_date = "2020-02-01"
end_date ="2021-10-31"
start_end = [start_date, end_date]

# define the feature level and components
# the label for charting is called 'name'
this_feature = {'slug':'les-alpes', 'name':"Alpen und Jura", 'level':'river_bassin'}

# these are the smallest aggregated components
# choices are water_name_slug=lake or river, city or location at the scale of a river bassin 
# water body or lake maybe the most appropriate
this_level = 'location'

fd_kwargs ={
    "filename": "resources/combined_alps_iqaasl.csv",
    "feature_name": this_feature['slug'], 
    "feature_level": this_feature['level'], 
    "these_features": this_feature['slug'], 
    "component": this_level, 
    "columns": columns, 
    "language": language, 
    "unit_label": unit_label, 
    "fail_rate": fail_rate,
    "code_data":dxCodes,
    "date_range": start_end,
    "water_type": None,    
}

fdx = featuredata.Components(**fd_kwargs)

# call the reports and languages
fdx.adjustForLanguage()
fdx.makeFeatureData()
fdx.locationSampleTotals(columns=["loc_date", "location", "date"])
fdx.makeDailyTotalSummary()
fdx.materialSummary()
fdx.mostCommon()
# !this is the feature data!
fd = fdx.feature_data

# this maps the slug-value to the proper name
alpes_survey_locations = dfBeaches[dfBeaches.river_bassin == 'les-alpes']

#### Alpes municipal layer

In [11]:
data = fdx.sample_totals.copy()
data["river_bassin"] = 'les-alpes'

feature_map = alpes_survey_locations["water_name"]
data["feature"] = data.location.map(lambda x: feature_map.loc[x])

city_map = fd[["location", "city"]].drop_duplicates()
city_map.set_index("location", drop=True, inplace=True)
data["city"] = data.location.map(lambda x: city_map.loc[x, "city"])

alpes_municipal_layer = data.groupby(["city", "feature"], as_index=False).agg(agg_columns)

city_gps =  alpes_survey_locations[["city", "latitude", "longitude"]].drop_duplicates("city")
city_gps.set_index("city", inplace=True, drop=True)

alpes_municipal_layer["lat"] = alpes_municipal_layer.city.map(lambda x: city_gps.loc[x, "latitude"])
alpes_municipal_layer["lon"] = alpes_municipal_layer.city.map(lambda x: city_gps.loc[x, "longitude"])
alpes_municipal_layer["samples"] = alpes_municipal_layer.loc_date

alpes_municipal_layer.to_csv("resources/output/gis/alpes_municipal_layer.csv", index=False)
alpes_municipal_layer.head()

,city,feature,quantity,p/100 m,loc_date,lat,lon,samples
0,Airolo,Alpes Lépontines,224,595.0,1,46.514257,8.607884,1
1,Andermatt,Alpes Uranaises,24,24.0,1,46.618327,8.598803,1
2,Calanca,Alpes Lépontines,78,96.0,1,46.330718,9.120103,1
3,Châtel-Saint-Denis,Préalpes Fribourgeoises,118,243.0,1,46.512592,6.949421,1
4,Grindelwald,Alpes Bernoises,169,281.0,1,46.658523,8.065323,1


#### Alpes location layer

In [12]:
alpes_location_layer = data.groupby(["location", "feature", "city"], as_index=False).agg(agg_columns)
lat_lon = alpes_survey_locations[["latitude", "longitude"]]

alpes_location_layer["lat"] = alpes_location_layer.location.map(lambda x: lat_lon.loc[x, "latitude"])
alpes_location_layer["lon"] = alpes_location_layer.location.map(lambda x: lat_lon.loc[x, "longitude"])
alpes_location_layer['samples'] = alpes_location_layer.loc_date

alpes_location_layer.to_csv("resources/output/gis/alpes_location_layer.csv", index=False)

alpes_location_layer

,location,feature,city,quantity,p/100 m,loc_date,lat,lon,samples
0,airolo,Alpes Lépontines,Airolo,224,595.0,1,46.514257,8.607884,1
1,andermatt,Alpes Uranaises,Andermatt,24,24.0,1,46.618327,8.598803,1
2,cabanes-des-diablerets,Alpes Vaudoises,Ormont-Dessus,165,1375.0,1,46.338604,7.215525,1
3,charmey,Préalpes Fribourgeoises,Val-de-Charmey,174,580.0,1,46.622641,7.189341,1
4,crans-montana,Alpes Valaisannes,Lens,47,106.0,1,46.334214,7.479386,1
5,grindelwald,Alpes Bernoises,Grindelwald,169,281.0,1,46.658523,8.065323,1
6,la-berra,Préalpes Fribourgeoises,La Roche,34,58.0,1,46.677399,7.178738,1
7,la-robella,Massif du Jura,Val-de-Travers,47,59.0,1,46.875905,6.548577,1
8,la-tzoumaz,Alpes Valaisannes,Riddes,88,110.0,1,46.144287,7.232897,1
9,les-crosets,Alpes Valaisannes,Val-d'Illiez,518,549.0,1,46.183591,6.832916,1


### GIS Return

The Aare survey area was divided into 500 m hexagons (hex grid). The landcover values and road lengths were extracted from the SWISS TLM REGIO map layers and assigned to the corresponding hexagon, for each hexagon in the hex grid. Geometries that were invalid or could not be assigned a value were skipped. In total there were X records skipped out of. There were no skipped hexagons that contained a survey location. That is to say all survey locations have a corresponding hexagon with land use and road length values.

#### The location-hex key

Each location has a corresponding hex_id. Other values can be assigned to the same hexagon. 

In [13]:
location_hex_key = pd.read_csv("resources/input/location_hex_key.csv")
location_key = location_hex_key[["hex_id", "location"]].set_index("hex_id")
location_key.head()

,location
hex_id,
54155,la-thiele_le-mujon_confluence
54590,pecos-plage
55478,signalpain
55921,la-petite-plage
65196,ruisseau-de-la-croix-plage


#### Road Length

The length for each "type" of road for each hex. The road length for the survey location *pecos-plage*

In [14]:
road_length = pd.read_csv("resources/input/road_length.csv")
road_length_aare = road_length[["hex_id", "OBJVAL", "MED", "length"]]
road_length["hex_id"] = road_length["hex_id"].astype("int")
road_length = road_length.groupby(["hex_id","OBJVAL"], as_index=False).length.sum()
road_length["variable"] = "roads"
road_length.rename(columns={"length":"value", "OBJVAL": "label"}, inplace=True)
road_length =road_length[["hex_id", "variable", "label", "value"]].copy()
road_length[road_length.hex_id == 54590]

,hex_id,variable,label,value
107,54590,roads,HauptStrAB6,385


__The different types of roads__

In [15]:
road_length.label.unique()

array(['NebenStr3', 'VerbindStr6', 'Fahrstraes', 'Autobahn',
       'HauptStrAB6', 'NebenStr6', 'Fussweg', 'VerbindStr4', 'Autob_Ri',
       'HauptStrAB4', 'Autostr'], dtype=object)

#### Land-cover

The land cover for the hexagon that contains *pecos-plage*

In [16]:
land_use = pd.read_csv("resources/input/land_use_aare.csv")
land_use["hex_id"] = land_use.hex_id.fillna(0)
land_use["hex_id"] = land_use.hex_id.astype(int)
cols = ["hex_id", "OBJVAL", "area"]
survey_land_use = land_use[cols].groupby(["hex_id", "OBJVAL"], as_index=False).area.sum()
survey_land_use["variable"] = "landcover"
survey_land_use.rename(columns={"area":"value", "OBJVAL": "label"}, inplace=True)
survey_land_use =survey_land_use[["hex_id", "variable", "label", "value"]].copy()
survey_land_use[survey_land_use.hex_id == 54590]

,hex_id,variable,label,value
78,54590,landcover,See,92293.560
79,54590,landcover,Siedl,67611.049


__The differnt types of landcover__

In [17]:
survey_land_use.label.unique()

array(['Fels', 'Geroell', 'Gletscher', 'Obstanlage', 'Reben', 'See',
       'Siedl', 'Wald', 'Sumpf', 'Stadtzentr', 'Stausee'], dtype=object)

In [18]:
survey_land_use[survey_land_use.hex_id == 51058]

,hex_id,variable,label,value
10,51058,landcover,Wald,93062.365


### Scaling the land use variables and survey results

The roads are given in meters and the land-cover in m². Both values will be scaled using the following: $ X{scaled} = \frac{X - X{min}}{X{max} - X{min}}$.

__Assumptions:__

The landuse proximal to a survey location has a direct impact on the type and quantity of objects found on the beach.

In [19]:
def scaleTheColumn(x, xmin, xmax):
    xscaled = (x-xmin)/(xmax-xmin)
    
    return xscaled

xmin = survey_land_use.value.min()
xmax = survey_land_use.value.max()
survey_land_use["scaled"] = survey_land_use.value.apply(lambda x: scaleTheColumn(x, xmin, xmax))

# redeine max min for the roads
xmin = road_length.value.min()
xmax = road_length.value.max()
road_length["scaled"] = road_length.value.apply(lambda x: scaleTheColumn(x, xmin, xmax))

astring= f"""
    The land-cover variable scaled: 
    
    {survey_land_use.scaled.describe()}
    """
print(astring)


    The land-cover variable scaled: 
    
    count    20372.000000
mean         0.481508
std          0.346974
min          0.000000
25%          0.157597
50%          0.439060
75%          0.808814
max          1.000000
Name: scaled, dtype: float64
    


In [20]:
astring= f"""
    The road_length variable scaled: 
    
    {road_length.scaled.describe()}
    """
print(astring)


    The road_length variable scaled: 
    
    count    10431.000000
mean         0.268084
std          0.138596
min          0.000000
25%          0.174084
50%          0.283377
75%          0.339660
max          1.000000
Name: scaled, dtype: float64
    


In [21]:
# the sample values:
mask = fdx.feature_data.river_bassin == "aare"
aare_samps = fdx.feature_data[mask].copy()
xmin = aare_samps["p/100 m"].min()
xmax = aare_samps["p/100 m"].max()
aare_samps["scaled"] = aare_samps["p/100 m"].apply(lambda x: scaleTheColumn(x, xmin, xmax))

astring= f"""
    The survey values scaled: 
    
    {aare_samps.scaled.describe()}
    """
print(astring)


    The survey values scaled: 
    
    count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: scaled, dtype: float64
    


### Fluvial inputs

The number, class, size and distance of the river instersections within 2km of the survey location.

In [22]:
intersections = pd.read_csv("resources/input/distance_to_intersection.csv")
intersections = intersections[["location",  "NAMN_2", "BREITE", "KLASSE", "OBJVAL", "OBJVAL_2", "distance"]]
intersections = intersections.drop_duplicates(["location", "NAMN_2","BREITE"])
intersections["distance"] = intersections.distance.astype(int)
intersections.rename(columns={"NAMN_2": "intersect", "BREITE":"size", "KLASSE": "klasse", "distance": "dist"}, inplace=True)
xmin = intersections.dist.min()
xmax = intersections.dist.max()

intersections["dist_scaled"] = intersections.dist.apply(lambda x: scaleTheColumn(x, xmin, xmax))

xmin = intersections.klasse.min()
xmax = intersections.klasse.max()
intersections["klasse_scaled"] = intersections.klasse.apply(lambda x: scaleTheColumn(x, xmin, xmax))

xmin = intersections.size.min()
xmax = intersections.size.max()
intersections["size_scaled"] = intersections["size"].apply(lambda x: scaleTheColumn(x, xmin, xmax))

intersections["k/s"] = intersections["klasse"]/intersections["size"]
xmin = intersections["k/s"].min()
xmax = intersections["k/s"].max()
intersections["sk_scaled"] = intersections["k/s"].apply(lambda x: scaleTheColumn(x, xmin, xmax))
intersections["effect"] = intersections["sk_scaled"]/intersections["dist"]
intersections = intersections.drop_duplicates(["location", "intersect"])
intersections[["location", "intersect", "size", "klasse",  "k/s", "effect"]][intersections.location == "pecos-plage"]

,location,intersect,size,klasse,k/s,effect
246,pecos-plage,Le Bey,9,9,1.0,0.000871
247,pecos-plage,La Thielle,6,6,1.0,0.000493
249,pecos-plage,La Brine,9,9,1.0,0.001016


In [23]:
intersections["klasse"]/intersections["size"]

0      1.000000
4      1.000000
5      0.777778
12     1.000000
13     0.444444
         ...   
234    0.444444
238    1.000000
246    1.000000
247    1.000000
249    1.000000
Length: 68, dtype: float64

In [24]:
6/4

1.5

In [25]:
# the cumulative effect of all the intersects for each location
intersections[["location", "intersect", "size", "klasse", "dist", "k/s", "effect"]][intersections.location == "wycheley"]

,location,intersect,size,klasse,dist,k/s,effect
186,wycheley,Aare,6,4,260,0.666667,0.001663
190,wycheley,Fulbach,9,9,843,1.000000,0.000962
194,wycheley,Kanal Aarboden,18,8,86,0.444444,0.002095


In [26]:
# the cumulative effect of all the intersects for each location
intersections[["location", "intersect", "size", "klasse",  "k/s", "dist", "effect"]][intersections.location == "oben-am-see"]

,location,intersect,size,klasse,k/s,dist,effect
226,oben-am-see,Aare,6,4,0.666667,292,0.001481
230,oben-am-see,Fulbach,9,9,1.000000,470,0.001725
234,oben-am-see,Kanal Aarboden,18,8,0.444444,480,0.000375


#### __Assumptions/Relation:__

1. There is a positive correlation between the size of the river or the class of the river and the quantity of trash it transports
2. The closer a survey location is to a river intersection point the more likely it is to receive trash from the intersection.
3. In general there is a negative correlation between a river intersection and the distance to any location on the lake.

__Background/Priors__

There is enough research that points to relations 1 and 2 (Seine maritime, Pay Bas). However, the contamination of remote Pacific Islands and Northern Europes coast line shows that even places far away from the source are seriously effected. The transport of plastics or more generally of trash on the Oceans is a well studied and documented process. A portion of the data is derived from _beach-litter-surveys_ conducted by a variety organizations. Trash has been monitored in the same fashion on Le Léman since November 2015. The combined effort has resulted in 312 samples from 145km of coastline. Furthermore, these samples were all conducted using the same protocol. 

As of 2020 the EU threshold for beach litter is 15 pieces of trash per 100 m (p/100m). The results for Switzerland in 2021 suggest a median value of 189 p/100m, the lowest values were in the Aare river catchment area and the Highest were in the Rhône catchment area (IQAASL). Recent sample data from Pays Bas have a mean of VALUE and median VALUE for VALUE samples. Relation one stems from the observations in Brienze and Bonnigen on the Brienzersee and Saint Gingolph on Lac Léman (both in Switzerland). These communities are located very near the mouth of major swiss rivers, Aare and Rhône respectively and are primarily rural communities. The amount of land that is étanche surrounding these surveys locations is low when compared to other communities in the survey. 

Lac Léman presents unique opportunities for researching the movement or deposition of trash on the shoreline. For example the resident time on the lake is 11 years, the water is moving slowly from Saint Gingolph to Geneva. There are not massive tidal events in relation to the ocean and there is an established culture and infrastructure dedicated to improving the conditions for the lakes wildlife. This includes litter campaigns, clean ups, communications and public education. Lac Léman is also urbanized, only 1% of the coastline is considered in a "natural state" and the real estate values are among the highest in the world.

__Enquiries__

1. Can the fluvial effect be used to predict trash values?
   1. Is there a higher probability of finding more trash at locations with a high fluvial effect?
   2. How likely are you to exceed the regional average in relation to the fluvial effect?
<br></br>
2. How does land use change the probability of finding an object?
   1. How does the probability of finding a given number of objects change when the land-use changes?
<br></br>
3. how does the probability of finding an object or a quantity of a group of objects change with location or region?
   1. Does the knowledge of the land-use change the expected result?
   2. If so, how reliable are the new assumptions?
   
The effects of land-use are calculated by using the meters squared (m²) of a 500 m hexagon that contains the sample location. This is a departure from the orginal method where the sample location was considered the reference point and the land use was considered as a function of the distance from the point. In this case a 500 m hex grid was made of Switzerland and the land use values for each 500 m hex were extracted from the Swss TLM Regio map. This set of maps has less detail than the first map but the data is complete with many relavent topographical categories labeled and classfied. For example the size and the class of the rivers on the map is detailed and defined in German and French. 

#### __Calculating Fluvial effects:__

__Definitions:__

* __size:__ german = "Kartografische“ Breite des Abschnitts, french = Largeur cartographique du tronçon, english = The size of the river at this section.

* __class:__ german = Breite, einheitlich über die ganze Länge, french = Importance, identique sur toute la longueur, english = The importance of the river, identical for the whole length.

The size value can vary depending on the location. the class variable is consistent for the whole variable. Note that the smaller the class the more important the river. The size attribute however reflects the actual size of the river at the point of intersection.




In [27]:
mask = (fdx.feature_data.river_bassin == "aare") & (aare_samples.w_t == "l")

aare_samples = fdx.feature_data[mask].copy()
aare_sample_totals = aare_samples.groupby(["loc_date", "city", "location"], as_index=False).pcs_m.sum()

def addEffect(x, effected):
    
    if x in effected.index:
        effect = effected.loc[x]
    else:
        effect = 0
    
    return effect

xmin = aare_sample_totals.pcs_m.min()
xmax = aare_sample_totals.pcs_m.max()
aare_sample_totals["scaled"] = aare_sample_totals.pcs_m.apply(lambda x: scaleTheColumn(x, xmin, xmax))
aare_sample_totals["hydro_effect"] = aare_sample_totals.location.map(lambda x: addEffect(x, intersect_effect))
sns.scatterplot(aare_sample_totals, x="hydro_effect", y="scaled")

NameError: name 'aare_samples' is not defined

In [ ]:
aare_samples["hydro_effect"] = aare_samples.location.map(lambda x: addEffect(x, intersect_effect))

code="G89"

test_samples = aare_samples[aare_samples.code == code].copy()

xmin = test_samples.pcs_m.min()
xmax = test_samples.pcs_m.max()
test_samples["scaled"] = test_samples.pcs_m.apply(lambda x: scaleTheColumn(x, xmin, xmax))

sns.scatterplot(data=test_samples, x="hydro_effect", y="scaled")

In [ ]:
corr, a_p = stats.spearmanr(test_samples["hydro_effect"], test_samples["scaled"])

In [ ]:
corr

In [ ]:
a_p